# Simulate Ginkgo Data

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
pwd

'/home/jupyter/Jet_Stream'

In [29]:
import numpy as np
import torch
import pprint
import matplotlib.pyplot as plt
import sys
import pickle
import argparse
import logging
import os
import tensorflow.compat.v1 as tf

# from ginkgo import invMass_ginkgo
# from invMass_ginkgo import *
from invMass_ginkgo_node import *
# from ginkgo.utils import get_logger


# logger = get_logger(level = logging.WARNING)
# fh = logging.FileHandler('spam.log')
# fh.setLevel(logging.WARNING)
# logger.addHandler(fh)

In [5]:
class ginkgo_simulator():
    def __init__(self,
                 rate,
                 pt_cut,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP,
                 root_rate= 1.5,
                ):
        
        self.root_rate = root_rate
        self.rate = rate
        self.pt_cut = pt_cut
        self.M2start = torch.tensor(M2start) # mass squared to start with
        self.Nsamples = Nsamples
        self.minLeaves = minLeaves
        self.maxLeaves = maxLeaves
        self.maxNTry = maxNTry
        self.jetType = jetType # W or QCD 
        self.jetM = np.sqrt(M2start) # mass to start with
        self.jetdir = np.array([1,1,1])
        self.jetP = jetP
        self.jetvec = self.jetP * self.jetdir / np.linalg.norm(self.jetdir)
        self.jet4vec = np.concatenate(([np.sqrt(self.jetP ** 2 + self.jetM ** 2)], self.jetvec))
        # logger.debug(f"jet4vec = {self.jet4vec}")
        
        if jetType == "W":
            # defined in paper, W jets have a different root rate
            self.rate=torch.tensor([self.root_rate,self.rate])
        elif jetType == "QCD":
            # QCD jets maintain the same rate throughout
            self.rate=torch.tensor([self.rate,self.rate])
        else:
            raise ValueError("Choose a valid jet type between W or QCD")



    def simulator(self):

        simulator = Simulator(jet_p = self.jet4vec,
                                         pt_cut = float(self.pt_cut),
                                         Delta_0 = self.M2start,
                                         M_hard = self.jetM ,
                                         num_samples = int(self.Nsamples),
                                         minLeaves = int(self.minLeaves),
                                         maxLeaves = int(self.maxLeaves),
                                         maxNTry = int(self.maxNTry)
                                         )
        return simulator
       
    def generate(self):
        
        simulator = self.simulator()
        jet_list = simulator(self.rate)

        # logger.debug(f"---"*10)
        # logger.debug(f"jet_list = {jet_list}")
        
        return jet_list

## Parameters

In [6]:
rate2 = torch.tensor(8.) # is this used?
# Parameters to get ~<10 constituents to test the trellis algorithm
pt_min = torch.tensor(4.**2)
### Physics inspired parameters to get ~ between 20 and 50 constituents
W_rate = 3.
QCD_rate = 1.5
QCD_mass = 30.

In [7]:
# the range of leaves that you would consider as valid generations
minLeaves = 8 #3
maxLeaves = 11 #100
# number of jets you wish to generate
Nsamples = 1
# exponential rate parameter
rate = 1.5
# mass squared cut off to yield leaves
pt_cut =  torch.tensor(1.1**2)
# mass squared to start with
# M2start = 80.**2
M2start = 10.**2
# the maximum times you are willing to try to get Nsamples
maxNTry = 1
# jetP=400.
jetP=4.

In [18]:
jetType ="QCD"

def nodeSum(node):
    if not node:
        return 0
    return node.logLH + nodeSum(node.left) + nodeSum(node.right)

ginkgo = ginkgo_simulator(
                 rate,
                 pt_cut ,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP)

jet_list = []
for i in range(2):
    print(f"Sample {i}")
    QCD_jets = ginkgo.generate()
    print("Number of nodes: " , len(QCD_jets))
    QCD_jets[0].llhSum = nodeSum(QCD_jets[0])
    print(QCD_jets[0].llhSum)
    #if len(QCD_jets) == 21:
    jet_list.append(QCD_jets)


Sample 0
Node 0
 Vec4: [10.77032961  2.30940108  2.30940108  2.30940108]
 Decay Rate: 1
 Mass Squared: tensor(100.)
 Log Likelihood: -8.343492298879692
 DIJ List: -8.343492298879692 0.4058080604871134 6.2180457198953585 2.6976042179282573

Node 1
 Vec4: [ 0.76938436 -0.54658424 -0.36753286  0.07284879]
 Decay Rate: tensor(0.1328)
 Mass Squared: tensor(0.1528)
 Log Likelihood: 0
 DIJ List:

Node 2
 Vec4: [10.00094525  2.85598532  2.67693394  2.23655228]
 Decay Rate: tensor(0.7969)
 Mass Squared: tensor(79.6941)
 Log Likelihood: -8.145483284750053
 DIJ List: -8.145483284750053 0.21895900973474589 2.573883307201682 1.6829860779701085

Node 3
 Vec4: [8.86249938 2.87466492 1.86852712 2.80779836]
 Decay Rate: tensor(0.7391)
 Mass Squared: tensor(58.9051)
 Log Likelihood: -7.991703301229545
 DIJ List: -7.991703301229545 0.08575627687418498 1.0156853948042104 0.00016964675240789778

Node 4
 Vec4: [ 1.1384456  -0.01867968  0.80840674 -0.57124614]
 Decay Rate: tensor(0.2014)
 Mass Squared: tenso

In [9]:
len(jet_list)

2

In [19]:
def make_datadict(jet):
    min_log_lik = 0
    n = len(jet)//2 + 1
    vec4s = np.zeros([n,4])
    leaves = []
    
    
    i = 0
    for node in jet:
        if node.logLH < min_log_lik:
            min_log_lik = node.logLH
        if node.left is None and node.right is None:
            vec4s[i] = node.vec4
            i += 1
            leaves.append(node)
    ddict = {
        'data' : vec4s,
        'llh': jet[0].llhSum,
        }
    print("total nodes: ", len(jet), "leaf nodes: " , i, "ground truth likelihood: " , min_log_lik)
    return ddict

In [20]:
#jet = jet_list[2]
#print(len(jet))

ddict = {}

sum = 0 
for i,jet in enumerate(jet_list):
    ddict[i] = make_datadict(jet)
    sum += ddict[i]['llh']
    print(ddict[i]['llh'])
    #print(ddict[i]['ground_truth_likelihood'])
print(sum/len(jet_list))

total nodes:  15 leaf nodes:  8 ground truth likelihood:  -8.74057939604766
-43.940515544920444
total nodes:  17 leaf nodes:  9 ground truth likelihood:  -10.110156327047417
-40.289345260925835
-42.114930402923136


In [69]:
ddict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [21]:
with open('withSum_2jets.p', 'wb') as handle:
    pickle.dump(ddict, handle)

In [23]:
jet_list[0][0].left

In [52]:
len(jet_list[0])

15

In [26]:
jet_list[0][0].llhSum

-43.940515544920444

In [27]:
jet_list[0][0].vec4

array([10.77032961,  2.30940108,  2.30940108,  2.30940108])

In [65]:

import random

# Example list of numbers
numbers = [1, 4, 5, 8, 10, 12, 13, 14]


# Sample two numbers from the list
samp0 = random.sample(numbers, 2)
samp1 = random.sample(numbers, 2)
samp2 = random.sample(numbers, 2)


with tf.Session() as sess:
    print(samp0)
    print(samp1)
    print(samp2)
    
    print(jet_list[0][samp0[0]].vec4)
    print(jet_list[0][samp0[1]].vec4)
    print()
    print(jet_list[0][samp1[0]].vec4)
    print(jet_list[0][samp1[1]].vec4)
    print()
    print(jet_list[0][samp2[0]].vec4)
    print(jet_list[0][samp2[1]].vec4)
    
    n_particles = 3 # K, the length of the vertical vector.
    n_leaves = 3 # numebr of leaves

    # valid pair 1
    y = tf.constant(list(jet_list[0][samp0[0]].vec4), dtype = tf.float64)
    z = tf.constant(list(jet_list[0][samp0[1]].vec4), dtype = tf.float64)

    # valid pair 2
    y0 = tf.constant(list(jet_list[0][samp1[0]].vec4), dtype = tf.float64)
    z0 = tf.constant(list(jet_list[0][samp1[1]].vec4), dtype = tf.float64)

    # invalid pair 1
    y1 = tf.constant(list(jet_list[0][samp2[0]].vec4), dtype = tf.float64)
    z1 = tf.constant(list(jet_list[0][samp2[1]].vec4), dtype = tf.float64)

    l_data_3x1x4 = tf.stack([y, y0, y1], axis=0)
    r_data_3x1x4 = tf.stack([z, z0, z1], axis=0)

    l_data_3x1x4 = tf.expand_dims(l_data_3x1x4, axis = 1)
    r_data_3x1x4 = tf.expand_dims(r_data_3x1x4, axis = 1)


    lam = tf.constant([1.5, 1.5, 1.5], dtype = tf.float64)

    lam = tf.reshape(lam, (3, 1))


    # Assign t_cut based on max mass of leaves
    t_cut = tf.constant(1.1**2, dtype = tf.float64)

    new_mtx_KxSxA, tl, tr, tp = llh_bc(l_data_3x1x4, r_data_3x1x4, t_cut, lam)
    a = new_mtx_KxSxA, tl, tr, tp
    # a = tl_log_prob(tl, tp, lam)
    # b = tr_log_prob(tr, tp, tl, lam)
    # c = [a,b]
    res = sess.run(a)
    # print(res.shape)
    print(res[0])

[12, 5]
[13, 4]
[4, 1]
[1.35424993 0.69765792 1.05831246 0.42194797]
[ 0.28934727 -0.01068778 -0.00726624  0.13143374]

[0.5055145  0.46379717 0.00504984 0.17431713]
[ 1.1384456  -0.01867968  0.80840674 -0.57124614]

[ 1.1384456  -0.01867968  0.80840674 -0.57124614]
[ 0.76938436 -0.54658424 -0.36753286  0.07284879]


InvalidArgumentError: Inner dimensions of output shape must match inner dimensions of updates shape. Output: [2,1] updates: [0]
	 [[node TensorScatterUpdate_166 (defined at opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'TensorScatterUpdate_166':
  File "opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1041, in launch_instance
    app.start()
  File "opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()

In [63]:
def llh_bc(l_data_Kx1x4, r_data_Kx1x4, t_cut, decay_factor_Kx1):

            # grab left invariant mass
            tL_Kx1 = l_data_Kx1x4[:, :,0] ** 2 - tf.norm(l_data_Kx1x4[:, :,1:], axis = -1) ** 2
            # grab right invariant mass
            tR_Kx1 = r_data_Kx1x4[:, :,0] ** 2 - tf.norm(r_data_Kx1x4[:, :,1:], axis = -1) ** 2

            p_data_Kx1x4 = l_data_Kx1x4 + r_data_Kx1x4

            tp_Kx1 = p_data_Kx1x4[:, :,0] ** 2 - tf.norm(p_data_Kx1x4[:, :,1:], axis = -1) ** 2

            is_negative_Kx1 = tf.logical_or(tf.logical_or(tf.less(tL_Kx1, 0), tf.less(tR_Kx1, 0)), tf.less_equal(tp_Kx1, 0))

            is_invalid_Kx1 = tf.logical_or(
                                tf.less_equal(tp_Kx1, t_cut),
                                tf.logical_or(
                                   tf.logical_or(
                                       tf.greater_equal(tL_Kx1, (1 - 1e-3) * tp_Kx1),
                                       tf.greater_equal(tR_Kx1, (1 - 1e-3) * tp_Kx1)
                                   ),
                                   tf.greater(
                                       tf.sqrt(tL_Kx1) + tf.sqrt(tR_Kx1),
                                       tf.sqrt(tp_Kx1)
                                   )
                                )
                             )


            def valid_calc(tp_Xx1, tL_Xx1, tR_Xx1, decay_factor_Xx1):

                def get_logp(tP_local_Xx1, t_Xx1, t_cut, decay_factor_Xx1):
                    """ Here we call the actual PDFs and CDFs defined in Eq (7) of the paper"""

                    def prob_is_leaf(tP_local_Yx1, t_Yx1, t_cut, decay_factor_Yx1):
                        """ The CDF defined in Eq (7) of the paper """
                        # Probability of the shower to stop F_s
                        one_minus_cdf = 1 - tf.math.exp(- (1 - 1e-3)*decay_factor_Yx1)
                        prob = - tf.math.log(one_minus_cdf)\
                               + tf.math.log(decay_factor_Yx1) - tf.math.log(tP_local_Yx1) - decay_factor_Yx1 * t_Yx1 / tP_local_Yx1
                        return prob

                    def prob_is_not_leaf(tP_local_Xx1, t_Xx1, t_cut, decay_factor_Xx1):
                        """ The PDF defined in Eq (7) of the paper """
                        t_upper_Xx1 = tf.minimum(tP_local_Xx1, t_cut) #There are cases where tp2 < t_cut
                        one_minus_cdf = 1 - tf.math.exp(- (1 - 1e-3) * decay_factor_Xx1)
                        prob = -tf.math.log(one_minus_cdf) + \
                                tf.math.log(1 - tf.math.exp(- decay_factor_Xx1 * t_upper_Xx1 / tP_local_Xx1))
                        return prob

                    results_Xx1 = prob_is_not_leaf(tP_local_Xx1, t_Xx1, t_cut, decay_factor_Xx1)
                    
                    any_satisfy = tf.reduce_any(tf.greater(t_Xx1, t_cut))
                    indices_Yx1 = tf.cond(any_satisfy,
                     lambda: tf.where(tf.greater(t_Xx1, t_cut)),
                     lambda: tf.constant([], dtype=tf.int64))
                    #indices_Yx1 = tf.where(tf.greater(tf.squeeze(t_Xx1), t_cut))

                    tP_local_new_Yx1 = tf.gather(tf.squeeze(tP_local_Xx1), indices_Yx1)
                    t_new_Yx1 = tf.gather(tf.squeeze(t_Xx1), indices_Yx1)
                    decay_factor_Yx1 = tf.gather(tf.squeeze(decay_factor_Xx1), indices_Yx1)

                    updates_Yx1 = prob_is_leaf(tP_local_new_Yx1, t_new_Yx1, t_cut, decay_factor_Yx1)

                    results_Xx1 = tf.tensor_scatter_nd_update(results_Xx1, indices_Yx1, updates_Yx1)

                    return results_Xx1

                # We always assign the left node as the node with the greater invariant mass for consistency
                # To do this, we find the invariant mass squared for each node as a function of the parent
                # by defining tpLR and tpRL using Eq (6) of the paper
                # this is something akin to the parent invarinat mass in case left is bigger than right and the
                # parent invariant mass in case right is bigger than left
                tpLR_Xx1 = (tf.sqrt(tp_Xx1) - tf.sqrt(tL_Xx1)) ** 2
                tpRL_Xx1 = (tf.sqrt(tp_Xx1) - tf.sqrt(tR_Xx1)) ** 2

                # Calculate the log propobability using the CDFs and PDFs
                # for each of the two cases where the left/right node is ultimately greater
                logpLR_Xx1 = tf.cast(tf.math.log(1/2), dtype = tf.float64) + get_logp(tp_Xx1, tL_Xx1, t_cut,decay_factor_Xx1) + get_logp(tpLR_Xx1, tR_Xx1, t_cut, decay_factor_Xx1) 

                logpRL_Xx1 = tf.cast(tf.math.log(1/2), dtype = tf.float64) + get_logp(tp_Xx1, tR_Xx1, t_cut, decay_factor_Xx1) + get_logp(tpRL_Xx1, tL_Xx1, t_cut, decay_factor_Xx1)

                # take the product of the two rightmost terms in Eq (8) where the one_minus_cdf term is distributed
                logp_split_Xx1 = tf.reduce_logsumexp(tf.stack([logpLR_Xx1, logpRL_Xx1]), axis = 0)

                # Add the term for the likelihood for sampling uniformly over a 2-sphere
                logLH_Xx1 = logp_split_Xx1 + tf.math.log(1 / (4 * tf.constant(np.pi, dtype = tf.float64)))

                return tf.cast(logLH_Xx1, dtype = tf.float64)


            results_Kx1 = -tf.float64.max * tf.cast(tf.ones_like(is_negative_Kx1), dtype=tf.float64)

            indices_Xx1 = tf.where(
                            tf.logical_and(
                                tf.logical_not(tf.squeeze(is_negative_Kx1)),
                                tf.logical_not(tf.squeeze(is_invalid_Kx1))
                            )
                          )

            tp_new_Xx1 = tf.gather(tf.squeeze(tp_Kx1), indices_Xx1)
            tL_new_Xx1 = tf.gather(tf.squeeze(tL_Kx1), indices_Xx1)
            tR_new_Xx1 = tf.gather(tf.squeeze(tR_Kx1), indices_Xx1)
            decay_factor_Xx1 = tf.gather(tf.squeeze(decay_factor_Kx1), indices_Xx1)

            updates_Xx1 = valid_calc(tp_new_Xx1, tL_new_Xx1, tR_new_Xx1, decay_factor_Xx1)

            results_Kx1 = tf.tensor_scatter_nd_update(results_Kx1, indices_Xx1, updates_Xx1)
            parent_vec4_Kx1x4 = tf.squeeze(l_data_Kx1x4 + r_data_Kx1x4)


            results_Kx1_copied = tf.tile(
                tf.reshape(
                    tf.squeeze(results_Kx1), (-1, 1)
                ), 
                (1, 4)
            )

            vec4_Kx4 = tf.squeeze(parent_vec4_Kx1x4)

            like_stacked_vec4_Kx2x4 = tf.stack(
                        [
                            results_Kx1_copied,
                            vec4_Kx4
                        ],
                        axis=1
                    )
            return like_stacked_vec4_Kx2x4, tL_Kx1, tR_Kx1, tp_Kx1